In [1]:
import ast
import matplotlib.pyplot as plt
import numpy as np
import os
import pandas as pd
import pickle
import random
import seaborn as sn
from keras.utils import to_categorical, plot_model, np_utils
from random import randrange
from sklearn.cluster import KMeans
from sklearn.model_selection import train_test_split
from sklearn.neighbors import KNeighborsClassifier
from sklearn.preprocessing import LabelEncoder, StandardScaler, LabelBinarizer

num_classes = 27

Using TensorFlow backend.
c:\users\cesar\appdata\local\programs\python\python37\lib\site-packages\tensorflow\python\framework\dtypes.py:516: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_qint8 = np.dtype([("qint8", np.int8, 1)])
c:\users\cesar\appdata\local\programs\python\python37\lib\site-packages\tensorflow\python\framework\dtypes.py:517: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_quint8 = np.dtype([("quint8", np.uint8, 1)])
c:\users\cesar\appdata\local\programs\python\python37\lib\site-packages\tensorflow\python\framework\dtypes.py:518: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_qint16 = np.dtype([("qint16", np.int16

In [6]:
def get_dataset(gerar_estatisticas=False):
    # Formata os arquivos em dicionário na forma:
    # {'a': [0...[0...[0...30]...100]...10]}
    # 30 dados (gx, gy, gz, ax, ay, az para 5 dedos)
    # com 100 amostras para cada sinal
    # e realizado 10 vezes

    directory = 'Dados_completos'
    letras = os.listdir(directory) 
    num_letras = len(letras)

    dataset = pd.DataFrame()

    files = {}
    for letra in letras:
        files[letra] = os.listdir(directory+'/'+letra)

    stats = []
    all_dataframe = {}

    for letra, value_list in files.items():
        for f in value_list:
            with open(directory+"/"+letra+"/"+f) as file:
                lines = [line.strip() for line in file]
                lines = ast.literal_eval(lines[0])
                test = pd.DataFrame(lines)
                test['label'] = letra
                dataset = dataset.append(test, ignore_index = True)
                
                if gerar_estatisticas:
                    d = {'media':test.mean(), 
                         'desvio_padrao': test.std(), 
                         'max': test.max(), 
                         'min': test.min()
                        }
                    stats.append(pd.DataFrame(d))

                    if letra not in all_dataframe:
                        all_dataframe[letra] = []
                    all_dataframe[letra].append(test)
    
    if gerar_estatisticas:
        if not os.path.exists('./estatisticas'):
            os.mkdir('./estatisticas')

        estatisticas = {}
        i = 0;
        for letr in letras:
            estatisticas[letr] = stats[i:i+10]
            i+=10

        for key,list_values in estatisticas.items():
            for i,v in enumerate(list_values):
                v.to_excel('./estatisticas/' + key+'_'+str(i)+'.xlsx')
                
    return dataset, all_dataframe, letras

def get_minidataset(dataset):
    directory = 'Dados_completos'
    letras = os.listdir(directory) 
    
    # mini_dataset_train = []
    mini_dataset_test = []
    nums_gerados = []
    mini_dataset_test = pd.DataFrame()
    mini_dataset_train = pd.DataFrame()
    for letr in letras:
        for i in range(10):
    #         aleat = randrange(100)+100*i
            aleat = [ num + 100*i for num in random.sample(range(100), 11) ]
            nums_gerados = nums_gerados + aleat
    #         mini_dataset_train.append(dataset[dataset.label==letr].iloc[aleat[0]])
            # 10*10 amostras para treino
            mini_dataset_train = mini_dataset_train.append(dataset[dataset.label==letr].iloc[aleat[:10]])
            # 1 * 10 amostras para teste
            mini_dataset_test = mini_dataset_test.append(dataset[dataset.label==letr].iloc[aleat[10]])

    mini_dataset_train = pd.DataFrame(mini_dataset_train).reset_index()
    mini_dataset_train = mini_dataset_train.drop('index', 1)

    mini_dataset_test = pd.DataFrame(mini_dataset_test).reset_index()
    mini_dataset_test = mini_dataset_test.drop('index', 1)

    # Apaga os dados de teste do dataset
    tmp = 0
    for i,num in enumerate(nums_gerados):
        if(i!= 0 and i%110==0):
            tmp += 1000
        dataset = dataset.drop(num+tmp)

    dataset = dataset.reset_index(drop=True)

    # mini_dataset_train.to_csv('mini_dataset_train.csv')
    # mini_dataset_test.to_csv('mini_dataset_test.csv')
    
# save_histogramas(all_dataframe, letras)
# get_estatisticas_por_letra(estatisticas, letras)

In [19]:
def gerar_dic_conv_hist():
    nome_dados = ['ax0', 'ay0', 'az0', 'gx0', 'gy0', 'gz0',
                 'ax1', 'ay1', 'az1', 'gx1', 'gy1', 'gz1',
                 'ax2', 'ay2', 'az2', 'gx2', 'gy2', 'gz2',
                 'ax3', 'ay3', 'az3', 'gx3', 'gy3', 'gz3',
                 'ax4', 'ay4', 'az4', 'gx4', 'gy4', 'gz4']

    dic_conv_hist = {}
    for i,nom in enumerate(nome_dados):
        dic_conv_hist[i] = nom
    return dic_conv_hist
        

def save_histogramas(all_dataframe, letras):
    if not os.path.exists('./histogramas'):
        os.mkdir('./histogramas')

    for letra in letras:
        if not os.path.exists('./histogramas/'+letra):
            os.mkdir('./histogramas/'+letra)
        for ex in range(10):
            if not os.path.exists('./histogramas/'+letra+"/"+str(ex)):
                os.mkdir('./histogramas/'+letra+"/"+str(ex))
        
    dic_conv_hist = gerar_dic_conv_hist()

    for letra in letras:
        for ex in range(10):
            direc = './histogramas/'+letra+"/"+ str(ex) + "/"
            for dado in range(30):
                figure = all_dataframe[letra][ex].iloc[:,dado].hist(bins=100, range=[-16384, 16383]).get_figure()
                plt.tight_layout()
                figure.savefig(direc + letra + "_" + dic_conv_hist[dado] + ".png")
                plt.clf()
                

def get_estatisticas_por_letra(estatisticas, letras):

    est_letra = {}
    for key,list_values in estatisticas.items():
        est_letra[key] = pd.DataFrame()
        for i,v in enumerate(list_values):
            est_letra[key] = est_letra[key].append(pd.DataFrame(v))
        est_letra[key] = est_letra[key].reset_index()

    est_tmp = {}
    dic_conv_hist = gerar_dic_conv_hist()

    for letra in letras:
        est_tmp[letra] = []
        for i in range(30):
            tmp_pd = est_letra[letra][i::30]
            dic_tmp = {'desvio_padrao': tmp_pd['desvio_padrao'].std(ddof=0), 
                       'max': tmp_pd['max'].max(), 
                       'min': tmp_pd['min'].min(),
                       'media': tmp_pd['media'].mean(),
#                        'label': dic_conv_hist[i]
                      }
            est_tmp[letra].append(pd.DataFrame(dic_tmp, index=[i]))
    
    if not os.path.exists('./estatisticas_gerais'):
        os.mkdir('./estatisticas_gerais')
    
    estatisticas_geral = {}
    for letra in letras:
        estatisticas_geral[letra] = pd.concat(est_tmp[letra])
        estatisticas_geral[letra].to_csv('./estatisticas_gerais/'+letra+'.csv')
        
    return estatisticas_geral

# Check num of samples of the same class
def get_index_letra(letra, _y):
    _lst_letra = []
    for i,y in enumerate(_y):
        if y.argmax()==letra:
            _lst_letra.append(i)
    return _lst_letra

In [20]:
dataset, all_dataframe, letras = get_dataset(False)

In [21]:
# save_histogramas(all_dataframe, letras)

<Figure size 432x288 with 0 Axes>

In [15]:
X = dataset.iloc[:,0:30] 
y = dataset.iloc[:,30] 

y = LabelEncoder().fit_transform(y)

X = X/16384
# 50% treino + 50% (val + test)
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size = 0.5, stratify=y) 
# 25% val e 25% test
X_test, X_val, y_test, y_val = train_test_split(X_test, y_test, test_size = 0.5, stratify=y_test)

X_train = X_train.reset_index().iloc[:,1:31]
X_test = X_test.reset_index().iloc[:,1:31]
X_val = X_val.reset_index().iloc[:,1:31]

# Tranform training labels to one-hot encoding
y_train = np_utils.to_categorical(y_train, num_classes)

# Tranform test labels to one-hot encoding
y_val = np_utils.to_categorical(y_val, num_classes)
y_test = np_utils.to_categorical(y_test, num_classes)

In [22]:
X_train

,0,1,2,3,4,5,6,7,8,9,...,20,21,22,23,24,25,26,27,28,29
0,0.363037,0.869141,-0.257080,-0.009460,0.007141,-0.000671,0.302734,0.884521,-0.383789,-0.036865,...,-0.995361,-0.014832,-0.014832,-0.007263,0.042480,0.955322,-0.123047,-0.026733,0.028259,-0.006958
1,-0.804688,-0.468018,0.391846,-0.038269,-0.018127,-0.017273,-0.129150,-0.731445,0.599854,-0.081482,...,0.825928,-0.030823,0.007996,0.024414,0.135498,0.506104,0.824951,-0.055298,0.032654,-0.000732
2,-0.276855,-0.964844,-0.111816,-0.013916,0.016663,0.019043,0.145264,0.925293,0.383545,-0.032654,...,0.791260,0.021362,-0.021851,-0.002197,0.204346,0.958496,0.127197,-0.014771,0.018860,0.002319
3,0.508545,0.880615,-0.007080,0.005798,0.003845,0.043030,0.278076,0.948975,-0.346191,0.007935,...,-0.730469,0.022400,-0.034363,0.023804,0.336426,-0.758545,-0.686035,0.005493,0.004333,0.026245
4,-0.832275,-0.472412,0.453613,-0.071960,0.058533,0.080444,-0.318359,-0.689209,0.649414,0.041260,...,0.463623,0.032898,-0.039612,0.055481,0.185791,-0.955811,0.229492,-0.011230,0.017639,0.055359
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
13495,0.595947,-0.658203,0.599609,-0.046753,0.016663,-0.013184,0.308350,-0.904053,0.119629,-0.071411,...,0.660400,-0.031677,-0.004700,-0.018250,0.246826,0.566895,0.763916,-0.044800,0.013428,-0.038147
13496,-0.284180,-0.959473,0.120361,-0.028748,0.016296,0.000854,0.251709,0.928223,0.323975,-0.053406,...,0.837891,-0.010132,-0.004639,0.008118,0.157715,0.959961,0.232422,-0.035706,0.015320,-0.003540
13497,-0.833252,-0.445068,0.361328,-0.026672,-0.064575,0.001892,-0.248047,-0.679443,0.637451,-0.083923,...,0.452393,-0.063171,0.015198,0.023315,0.172363,0.926025,0.315430,-0.087769,0.018799,-0.007996
13498,0.568848,-0.460693,0.820068,-0.038147,0.018433,0.005310,0.527832,-0.816162,0.114258,-0.039612,...,0.675293,-0.014038,-0.015808,0.004272,0.478027,0.340576,0.801025,-0.026917,0.009033,-0.013611


In [40]:
# X_train.to_csv('dataset_train.csv')
# X_val.to_csv('dataset_validation.csv')
# X_test.to_csv('dataset_test.csv')

# np.savetxt("labels_train.csv", y_train, delimiter=",")
# np.savetxt("labels_validation.csv", y_val, delimiter=",")
# np.savetxt("labels_test.csv", y_test, delimiter=",")

In [2]:
X_train = pd.read_csv("dataset_train.csv")
X_val = pd.read_csv('dataset_validation.csv')
X_test = pd.read_csv('dataset_test.csv')

X_train = X_train.iloc[:,1:31]
X_val = X_val.iloc[:,1:31]
X_test = X_test.iloc[:,1:31]

y_train = np.loadtxt("labels_train.csv", delimiter=",")
y_val = np.loadtxt("labels_validation.csv", delimiter=",")
y_test = np.loadtxt("labels_test.csv", delimiter=",")

In [7]:
X_total = X_train.append(X_test, ignore_index=True).append(X_val, ignore_index=True)

In [27]:
X_train.iloc[get_index_letra(4, y_train)]

,0,1,2,3,4,5,6,7,8,9,...,20,21,22,23,24,25,26,27,28,29
100,-0.807617,0.044678,0.610352,-0.013184,0.046021,-0.034485,-0.175049,0.449463,0.843750,0.019470,...,0.791504,0.013184,-0.021179,-0.044434,0.114990,0.813721,0.536865,0.004944,-0.004456,-0.046265
112,-0.905273,-0.030518,0.358887,-0.070557,-0.004578,-0.003357,-0.222168,0.628174,0.705322,-0.063660,...,0.600830,-0.029419,0.003723,0.005798,0.037842,0.917236,0.265869,-0.048157,0.024170,-0.014221
115,-0.765625,-0.111572,0.573975,0.017517,0.002319,-0.029114,-0.279053,0.420654,0.789307,-0.028015,...,0.841797,0.020203,-0.014343,0.013916,0.043945,0.582275,0.691650,0.014465,0.012024,0.015930
136,-0.818604,-0.031982,0.630615,-0.079712,0.011780,0.015991,-0.250488,0.384521,0.856934,-0.041382,...,0.829834,-0.006714,-0.002136,0.015015,0.075195,0.751465,0.609375,-0.026794,0.017944,-0.005554
146,-0.812500,-0.046875,0.583496,-0.000366,-0.003906,-0.014282,-0.316650,0.471680,0.791992,-0.061340,...,0.918457,-0.010071,-0.007385,0.011047,0.009766,0.676270,0.667480,-0.030273,0.007568,0.000488
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
13399,-0.842773,-0.057129,0.549561,-0.024536,-0.005676,-0.009216,-0.288086,0.431641,0.816406,-0.058777,...,0.809814,-0.024414,0.019531,-0.001648,0.025391,0.799316,0.537109,-0.038452,0.015747,-0.030701
13423,-0.840332,0.006592,0.574707,-0.037170,0.006042,-0.006165,-0.290283,0.459717,0.821289,-0.060669,...,0.803467,-0.016357,0.002991,0.007629,0.030029,0.823975,0.537598,-0.039001,0.016602,-0.008179
13463,-0.771484,0.041016,0.591797,-0.041260,-0.008423,-0.018311,-0.173096,0.401611,0.854004,-0.050476,...,0.792236,-0.011902,0.005249,-0.008240,0.108154,0.759277,0.555420,-0.028564,0.017883,-0.025330
13476,-0.740967,0.009033,0.769043,-0.062561,0.004639,0.014709,-0.176758,0.255859,0.935547,-0.064758,...,0.846924,-0.026489,-0.007996,0.015442,0.134033,0.719727,0.664062,-0.053894,0.006348,-0.011658


In [53]:
# Gerar K-Means de 1 a 270 com clusters aleatórios
for i in range(1,271):
    km = KMeans(n_clusters=i, verbose=0)
    km.fit(X_train)
    centroides = km.cluster_centers_
    
    if not os.path.exists('./k-means centroides/aleatorios'):
        os.mkdir('./k-means centroides/aleatorios')
    np.savetxt("./k-means centroides/aleatorios/"+ str(i) + "_clusters.csv", centroides, delimiter=",")

In [28]:
def duplicate(y, n):
    y = np.expand_dims(y,axis=0)
    y_ret = y
    for i in range(n-1):
        y_ret = np.concatenate((y_ret, y))
    return y_ret

for j in range(1,11):
    centroides = []
    y_train_total = []
    for i in range(27):
        km = KMeans(n_clusters=j, verbose=0)
        X_train_letra = X_train.iloc[get_index_letra(i, y_train)]
        km.fit(X_train_letra)
        if len(centroides)==0:
            centroides = km.cluster_centers_
        else:
            centroides = np.concatenate((centroides, km.cluster_centers_))
        _y_train = np_utils.to_categorical(i, 27)
        _y_train = duplicate(_y_train, j)
        if len(y_train_total) == 0:
            y_train_total = _y_train
        else:
            y_train_total = np.concatenate((y_train_total, _y_train))

    if not os.path.exists('./k-means centroides/por_letra'):
        os.mkdir('./k-means centroides/por_letra')
    np.savetxt("./k-means centroides/por_letra/"+ str((i+1)*j) + "_clusters.csv", centroides, delimiter=",")
    
    np.savetxt("./k-means centroides/por_letra/"+ str((i+1)*j) +"_labels_knn-kmeans.csv", y_train_total, delimiter=",")